In [29]:
import json
import pandas as pd
import numpy as np
import time
from typing import Union
from tqdm import tqdm

import openai
from openai import AsyncOpenAI
import asyncio
import nest_asyncio

### GPT API

In [36]:
# GPT API 호출 및 생성

def openai_error_handling(func, **kwargs):
    try:
        return func(**kwargs)
    except (openai.APIError, openai.RateLimitError, openai.APIConnectionError) as e:
        print(e)
        print('retrying...')
        time.sleep(2)
        return openai_error_handling(func, **kwargs)


class OpenAIModel() :
    def __init__(self, model) :
        super(OpenAIModel, self).__init__()
        self.model_name = model
        self.client = AsyncOpenAI(
                        api_key='<type your OpenAI API Key>',
                        organization='<type your OpenAI Organization Key>',
                    )
        self.api_key = '<type your OpenAI API Key>'
        self.org_key = '<type your OpenAI Organization Key>'

    async def get_response(self, input_text) :
        completion = await openai_error_handling(
                self.client.chat.completions.create, 
                model=self.model_name, 
                messages=[{"role": "assistant", "content": input_text}], 
                #max_tokens=700, n=1, 
                temperature=0
                )
        return completion.choices[0].message.content

In [37]:
def generation(model_name, prompt):
    nest_asyncio.apply()

    model = OpenAIModel(model_name)

    loop = asyncio.get_event_loop()
    response = loop.run_until_complete(model.get_response(prompt))

    return response

### Prompt

RAG를 거쳐 얻은 Prompt를 활용한 생성 결과 vs 활용하지 않은 생성 결과의 차이를 확인

In [38]:
template = '''
### instruction ###
I want to make an AI pet profile with a picture of my dog.
The given Input Query Text is the form of the AI pet profile that the user wants. 
Use the Example Prompt below to create a New Prompt that can be put into the generated AI model according to the purpose of the given Text.

### Input Text ###
User Query: {}
Example Prompt: {}
=========
New Prompt:
'''

In [39]:
# prompt의 괄호 내에 포함시킬 값
user_query = 'I want to make my pet under the desk with a lot of toys.'
example_prompt = "beautiful picture of stray caramel dog and a green and yellow flag, glowing colors, fantasy art, trending on artstation, cgsociety, nice composition, great fantasy mood, classic Disney mood"

In [40]:
# prompt에 괄호가 n개면 format 인자로 순서에 맞춰서 n개 넣어주면 됨
prompt = template.format(user_query, example_prompt)

In [41]:
response = generation('gpt-4o', prompt)

In [42]:
response

'### New Prompt ###\nadorable picture of a dog under a desk surrounded by a lot of colorful toys, warm and cozy atmosphere, vibrant colors, trending on artstation, cgsociety, playful and joyful mood, classic Disney style'

user_query = 'I want to make my pet under the desk with a lot of toys.'  

example_prompt = "beautiful picture of stray caramel dog and a green and yellow flag, glowing colors, fantasy art, trending on artstation, cgsociety, nice composition, great fantasy mood, classic Disney mood"  

New Prompt = adorable picture of a dog under a desk surrounded by a lot of colorful toys, warm and cozy atmosphere, vibrant colors, trending on artstation, cgsociety, playful and joyful mood, classic Disney style